In [2]:
import os.path as osp
import time

import numpy as np
import pickle as pkl
import scipy.sparse as sp
import networkx as nx
import torch

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid, KarateClub
from torch_geometric.nn import GAE, VGAE, GCNConv
from torch_geometric.utils import to_networkx

import torch.nn.functional as F
from torch.nn.modules.module import Module
import torch.nn as nn
import torch.nn.modules.loss
from torch.nn.parameter import Parameter
from torch import optim

from sklearn.metrics import roc_auc_score, average_precision_score

In [5]:
def load_dataset(data_dir, dataset_cls, dataset_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToDevice(device)
    ])
    obj = dataset_cls(transform=transform) # dataset_cls(data_dir, dataset_name, transform=transform)
    network = to_networkx(obj[0])
    return nx.adjacency_matrix(network), obj[0].x

In [6]:
# adj, features = load_dataset(
#     osp.join('data', 'Planetoid'),
#     Planetoid,
#     'Cora'
# )

adj, features = load_dataset(
    osp.join('data', 'KarateClub'),
    KarateClub,
    'KarateClub'
)

In [8]:
print("Shape of Adj. Matrix: {}".format(adj.shape))
print("Shape of features: {}".format(tuple(features.shape)))

Shape of Adj. Matrix: (34, 34)
Shape of features: (34, 34)


In [9]:
class GCNLayer(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, dropout=0., act=F.relu):
        super(GCNLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.act = act
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, input_tensor, adj):
        input_tensor = F.dropout(input_tensor, self.dropout, self.training)
        support = torch.mm(input_tensor, self.weight)
        output_tensor = torch.spmm(adj, support)
        output_tensor = self.act(output_tensor)
        return output_tensor

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [10]:
def loss_function(preds, labels, mu, logvar, n_nodes, norm, pos_weight):
    print(type(pos_weight))
    cost = norm * F.binary_cross_entropy_with_logits(preds, labels, pos_weight=torch.tensor(pos_weight))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 / n_nodes * torch.mean(torch.sum(
        1 + 2 * logvar - mu.pow(2) - logvar.exp().pow(2), 1))
    return cost + KLD

In [11]:
class InnerProductDecoder(Module):
    """Decoder for using inner product for prediction."""

    def __init__(self, dropout, act=torch.sigmoid):
        super(InnerProductDecoder, self).__init__()
        self.dropout = dropout
        self.act = act

    def forward(self, z):
        z = F.dropout(z, self.dropout, training=self.training)
        adj = self.act(torch.mm(z, z.t()))
        return adj

In [12]:
class GCNModelVAE(Module):
    def __init__(self, input_feat_dim, hidden_dim1, hidden_dim2, dropout):
        super(GCNModelVAE, self).__init__()
        self.gc1 = GCNLayer(input_feat_dim, hidden_dim1, dropout, act=F.relu)
        self.gc2 = GCNLayer(hidden_dim1, hidden_dim2, dropout, act=lambda x: x)
        self.gc3 = GCNLayer(hidden_dim1, hidden_dim2, dropout, act=lambda x: x)
        self.dc = InnerProductDecoder(dropout, act=lambda x: x)

    def encode(self, x, adj):
        hidden1 = self.gc1(x, adj)
        return self.gc2(hidden1, adj), self.gc3(hidden1, adj)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x, adj):
        mu, logvar = self.encode(x, adj)
        z = self.reparameterize(mu, logvar)
        return self.dc(z), mu, logvar

In [13]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    # return sparse_to_tuple(adj_normalized)
    return sparse_mx_to_torch_sparse_tensor(adj_normalized)

In [14]:
def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def mask_test_edges(adj):
    """
    Function to build test set with 10% positive links
    """
    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0
    
    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 10.))
    num_val = int(np.floor(edges.shape[0] / 20.))
    
    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)
    
    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)
    
    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])
    
    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges):
            continue
        if ismember([idx_j, idx_i], train_edges):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])
    
    assert ~ismember(test_edges_false, edges_all)
    assert ~ismember(val_edges_false, edges_all)
    assert ~ismember(val_edges, train_edges)
    assert ~ismember(test_edges, train_edges)
    assert ~ismember(val_edges, test_edges)

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    # NOTE: these edge lists only contain single direction of edge!
    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false

def get_roc_score(emb, adj_orig, edges_pos, edges_neg):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    adj_rec = np.dot(emb, emb.T)
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

In [33]:
args = {
    'seed' : 42, 
    'epochs' : 200,
    'hidden1' : 8,
    'hidden2' : 4,
    'lr' : 0.01,
    'dropout' : 0.,
    'dataset_str' : 'KarateClub'
}

In [34]:
def build_model(args):
    print("Using {} dataset".format(args['dataset_str']))
    # adj, features = load_dataset(
    #     osp.join('data', 'Planetoid'),
    #     Planetoid,
    #     args['dataset_str']
    # )
    adj, features = load_dataset(
        osp.join('data', 'KarateClub'),
        KarateClub,
        'KarateClub'
    )
    n_nodes, feat_dim = tuple(features.shape)

    # Store original adjacency matrix (without diagonal entries) for later
    adj_orig = adj
    adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
    adj_orig.eliminate_zeros()
    
    adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
    adj = adj_train
    
    # Some preprocessing
    adj_norm = preprocess_graph(adj)
    adj_label = adj_train + sp.eye(adj_train.shape[0])
    # adj_label = sparse_to_tuple(adj_label)
    adj_label = torch.FloatTensor(adj_label.toarray())
    
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

    model = GCNModelVAE(feat_dim, args['hidden1'], args['hidden2'], args['dropout'])
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    
    hidden_emb = None
    for epoch in range(args['epochs']):
        t = time.time()
        model.train()
        optimizer.zero_grad()
        recovered, mu, logvar = model(features, adj_norm)
        loss = loss_function(preds=recovered, labels=adj_label,
                             mu=mu, logvar=logvar, n_nodes=n_nodes,
                             norm=norm, pos_weight=pos_weight)
        loss.backward()
        cur_loss = loss.item()
        optimizer.step()

        hidden_emb = mu.data.numpy()
        roc_curr, ap_curr = get_roc_score(hidden_emb, adj_orig, val_edges, val_edges_false)

        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cur_loss),
              "val_ap=", "{:.5f}".format(ap_curr),
              "time=", "{:.5f}".format(time.time() - t)
              )
    
    print("Optimization Finished!")

    roc_score, ap_score = get_roc_score(hidden_emb, adj_orig, test_edges, test_edges_false)
    print('Test ROC score: ' + str(roc_score))
    print('Test AP score: ' + str(ap_score))
    
    return model

In [35]:
model = build_model(args)

Using KarateClub dataset
<class 'numpy.float64'>
Epoch: 0001 train_loss= 1.20458 val_ap= 0.75556 time= 0.00567
<class 'numpy.float64'>
Epoch: 0002 train_loss= 1.11300 val_ap= 0.75556 time= 0.00406
<class 'numpy.float64'>
Epoch: 0003 train_loss= 1.00502 val_ap= 0.75556 time= 0.00310
<class 'numpy.float64'>
Epoch: 0004 train_loss= 0.98628 val_ap= 0.75556 time= 0.00315
<class 'numpy.float64'>
Epoch: 0005 train_loss= 1.07158 val_ap= 0.75556 time= 0.00331
<class 'numpy.float64'>
Epoch: 0006 train_loss= 1.07066 val_ap= 0.75556 time= 0.00333
<class 'numpy.float64'>
Epoch: 0007 train_loss= 1.21783 val_ap= 0.75556 time= 0.00317
<class 'numpy.float64'>
Epoch: 0008 train_loss= 1.09525 val_ap= 0.75556 time= 0.00327
<class 'numpy.float64'>
Epoch: 0009 train_loss= 1.08576 val_ap= 0.75556 time= 0.00329
<class 'numpy.float64'>
Epoch: 0010 train_loss= 0.98119 val_ap= 0.75556 time= 0.00387
<class 'numpy.float64'>
Epoch: 0011 train_loss= 0.92431 val_ap= 0.75556 time= 0.00261
<class 'numpy.float64'>
Epoch